In [1]:
import sklearn
import numpy as np
import pandas as pd
import xgboost as xgb
import tensorflow as tf


Diabetes data set

https://scikit-learn.org/stable/datasets/toy_dataset.html#diabetes-dataset

In [2]:
from sklearn import datasets

diabetes = datasets.load_diabetes()
X = diabetes.data
y = diabetes.target

In [63]:
X.shape

(442, 10)

In [4]:
y.shape

(442,)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2, random_state=42)

# Linear Regression With SKLearn

In [9]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression().fit(X_train, y_train)

In [17]:
from sklearn.metrics import mean_squared_error
import math

y_validation_hat = lr_model.predict(X_validation)
val_rmse = math.sqrt(mean_squared_error(y_validation, y_validation_hat))
y_train_hat = lr_model.predict(X_train)
train_rmse = math.sqrt(mean_squared_error(y_train, y_train_hat))


print("RMSE on training data: {}".format(train_rmse))
print("RMSE on validation data: {}".format(val_rmse))

RMSE on training data: 53.55881425327268
RMSE on validation data: 53.8532569849144


# Gradient Boost Tree with Xgboost

In [26]:
from xgboost import XGBRegressor

my_model = XGBRegressor(
    objective='reg:squarederror',
    learning_rate=0.1,
    max_depth=3,
    gamma=10,
    min_child_weight=10,
    n_estimators=200, 
#     subsample=0.87,
    colsample_bynode=1,
#     colsample_bytree=0.9,
    reg_alpha=0,
    reg_lambda=3,
    
    booster='gbtree',
    tree_method='hist',
    missing=None,
    verbosity=1
)

print(my_model.get_xgb_params())

trained = my_model.fit(
    X_train, y_train, early_stopping_rounds=10,
    # the last metric is used for early stopping
    eval_metric=["rmse"],
    eval_set=[(X_train, y_train), (X_validation, y_validation)], verbose=True
)


{'objective': 'reg:squarederror', 'base_score': None, 'booster': 'gbtree', 'colsample_bylevel': None, 'colsample_bynode': 1, 'colsample_bytree': None, 'gamma': 10, 'gpu_id': None, 'interaction_constraints': None, 'learning_rate': 0.1, 'max_delta_step': None, 'max_depth': 3, 'min_child_weight': 10, 'monotone_constraints': None, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': 0, 'reg_lambda': 3, 'scale_pos_weight': None, 'subsample': None, 'tree_method': 'hist', 'validate_parameters': None, 'verbosity': 1}
[0]	validation_0-rmse:157.33365	validation_1-rmse:148.61986
[1]	validation_0-rmse:144.35011	validation_1-rmse:136.29663
[2]	validation_0-rmse:132.74573	validation_1-rmse:125.11211
[3]	validation_0-rmse:122.43217	validation_1-rmse:115.12044
[4]	validation_0-rmse:113.20927	validation_1-rmse:106.47181
[5]	validation_0-rmse:104.99681	validation_1-rmse:98.47568
[6]	validation_0-rmse:97.72016	validation_1-rmse:91.62320
[7]	validation_0-rmse:91.27415	validation_1

# Linear Regression In Neural Network Style

In [54]:
import tensorflow as tf
from tensorflow import keras as keras
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Reshape, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import MeanSquaredError



In [66]:
def build_linear_regression():
    inputs = Input(shape=(10,))
    l = inputs    
    l = Dense(1, activation="linear", name='fc')(l)
    linear_regression_model = Model(inputs=inputs, outputs=l)
    print(linear_regression_model.summary())
    return linear_regression_model

def model_train(X_train, y_train, X_val, y_val, model):
    print(model.summary())
    early_stop = EarlyStopping(
        monitor="val_loss", min_delta=0, patience=3,
        verbose=0, mode="auto", baseline=None, restore_best_weights=True
    )
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=2, min_lr=0.05)

    callbacks_list = [early_stop, reduce_lr]

    # optimizer: gradient descent implementation
    adam_wn = Adam(learning_rate=0.8)
    
    # compile model with loss and optimizer; loss function needs to match the intention and the output of the last layer
    model.compile(
        loss="mean_squared_error", optimizer=adam_wn,
        metrics=[MeanSquaredError()]
    )
    
    # kick off model training
    return model.fit(
        x=X_train, y=y_train, validation_data=(X_val, y_val), batch_size=442,
        epochs=10000, verbose=1, shuffle=True,
        callbacks=callbacks_list)


In [64]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train)

In [65]:
X_train_scaled = scaler.transform(X_train)
X_validation_scaled = scaler.transform(X_validation)

In [ ]:

my_lr_model = build_linear_regression()

model_train(X_train_scaled, y_train, X_validation_scaled, y_validation, my_lr_model)

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 10)]              0         
_________________________________________________________________
fc (Dense)                   (None, 1)                 11        
Total params: 11
Trainable params: 11
Non-trainable params: 0
_________________________________________________________________
None
Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 10)]              0         
_________________________________________________________________
fc (Dense)                   (None, 1)                 11        
Total params: 11
Trainable params: 11
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10000
1/1 [===================

Epoch 40/10000
1/1 [==============================] - 0s 17ms/step - loss: 18160.5645 - mean_squared_error: 18160.5645 - val_loss: 19244.4668 - val_mean_squared_error: 19244.4668
Epoch 41/10000
1/1 [==============================] - 0s 22ms/step - loss: 17971.2969 - mean_squared_error: 17971.2969 - val_loss: 19047.3691 - val_mean_squared_error: 19047.3691
Epoch 42/10000
1/1 [==============================] - 0s 22ms/step - loss: 17783.6270 - mean_squared_error: 17783.6270 - val_loss: 18849.0918 - val_mean_squared_error: 18849.0918
Epoch 43/10000
1/1 [==============================] - 0s 17ms/step - loss: 17597.6699 - mean_squared_error: 17597.6699 - val_loss: 18649.8359 - val_mean_squared_error: 18649.8359
Epoch 44/10000
1/1 [==============================] - 0s 17ms/step - loss: 17413.5176 - mean_squared_error: 17413.5176 - val_loss: 18449.7832 - val_mean_squared_error: 18449.7832
Epoch 45/10000
1/1 [==============================] - 0s 17ms/step - loss: 17231.2168 - mean_squared_erro

Epoch 86/10000
1/1 [==============================] - 0s 17ms/step - loss: 11226.3682 - mean_squared_error: 11226.3682 - val_loss: 11841.8145 - val_mean_squared_error: 11841.8145
Epoch 87/10000
1/1 [==============================] - 0s 16ms/step - loss: 11110.4932 - mean_squared_error: 11110.4932 - val_loss: 11724.7148 - val_mean_squared_error: 11724.7148
Epoch 88/10000
1/1 [==============================] - 0s 17ms/step - loss: 10995.9150 - mean_squared_error: 10995.9150 - val_loss: 11608.6465 - val_mean_squared_error: 11608.6465
Epoch 89/10000
1/1 [==============================] - 0s 17ms/step - loss: 10882.6182 - mean_squared_error: 10882.6182 - val_loss: 11493.6309 - val_mean_squared_error: 11493.6309
Epoch 90/10000
1/1 [==============================] - 0s 16ms/step - loss: 10770.5898 - mean_squared_error: 10770.5898 - val_loss: 11379.6807 - val_mean_squared_error: 11379.6807
Epoch 91/10000
1/1 [==============================] - 0s 17ms/step - loss: 10659.8193 - mean_squared_erro

1/1 [==============================] - 0s 16ms/step - loss: 7081.6743 - mean_squared_error: 7081.6743 - val_loss: 7539.7544 - val_mean_squared_error: 7539.7544
Epoch 133/10000
1/1 [==============================] - 0s 17ms/step - loss: 7015.2231 - mean_squared_error: 7015.2231 - val_loss: 7470.2485 - val_mean_squared_error: 7470.2485
Epoch 134/10000
1/1 [==============================] - 0s 16ms/step - loss: 6949.6401 - mean_squared_error: 6949.6401 - val_loss: 7401.6030 - val_mean_squared_error: 7401.6030
Epoch 135/10000
1/1 [==============================] - 0s 16ms/step - loss: 6884.9185 - mean_squared_error: 6884.9185 - val_loss: 7333.8105 - val_mean_squared_error: 7333.8105
Epoch 136/10000
1/1 [==============================] - 0s 16ms/step - loss: 6821.0488 - mean_squared_error: 6821.0488 - val_loss: 7266.8604 - val_mean_squared_error: 7266.8604
Epoch 137/10000
1/1 [==============================] - 0s 16ms/step - loss: 6758.0205 - mean_squared_error: 6758.0205 - val_loss: 7200.7

Epoch 179/10000
1/1 [==============================] - 0s 16ms/step - loss: 4769.6284 - mean_squared_error: 4769.6284 - val_loss: 5087.5688 - val_mean_squared_error: 5087.5688
Epoch 180/10000
1/1 [==============================] - 0s 16ms/step - loss: 4735.6665 - mean_squared_error: 4735.6665 - val_loss: 5050.9727 - val_mean_squared_error: 5050.9727
Epoch 181/10000
1/1 [==============================] - 0s 16ms/step - loss: 4702.2261 - mean_squared_error: 4702.2261 - val_loss: 5014.9131 - val_mean_squared_error: 5014.9131
Epoch 182/10000
1/1 [==============================] - 0s 16ms/step - loss: 4669.3022 - mean_squared_error: 4669.3022 - val_loss: 4979.3857 - val_mean_squared_error: 4979.3857
Epoch 183/10000
1/1 [==============================] - 0s 16ms/step - loss: 4636.8882 - mean_squared_error: 4636.8882 - val_loss: 4944.3818 - val_mean_squared_error: 4944.3818
Epoch 184/10000
1/1 [==============================] - 0s 16ms/step - loss: 4604.9775 - mean_squared_error: 4604.9775 - 

...

Epoch 272/10000
1/1 [==============================] - 0s 15ms/step - loss: 3162.1306 - mean_squared_error: 3162.1306 - val_loss: 3296.2451 - val_mean_squared_error: 3296.2451
Epoch 273/10000
1/1 [==============================] - 0s 15ms/step - loss: 3155.7349 - mean_squared_error: 3155.7349 - val_loss: 3288.5039 - val_mean_squared_error: 3288.5039
Epoch 274/10000
1/1 [==============================] - 0s 15ms/step - loss: 3149.4670 - mean_squared_error: 3149.4670 - val_loss: 3280.9033 - val_mean_squared_error: 3280.9033
Epoch 275/10000
1/1 [==============================] - 0s 16ms/step - loss: 3143.3271 - mean_squared_error: 3143.3271 - val_loss: 3273.4434 - val_mean_squared_error: 3273.4434
Epoch 276/10000
1/1 [==============================] - 0s 16ms/step - loss: 3137.3127 - mean_squared_error: 3137.3127 - val_loss: 3266.1196 - val_mean_squared_error: 3266.1196
Epoch 277/10000
1/1 [==============================] - 0s 16ms/step - loss: 3131.4207 - mean_squared_error: 3131.4207 - 

1/1 [==============================] - 0s 16ms/step - loss: 2972.0408 - mean_squared_error: 2972.0408 - val_loss: 3055.8752 - val_mean_squared_error: 3055.8752
Epoch 319/10000
1/1 [==============================] - 0s 16ms/step - loss: 2969.6743 - mean_squared_error: 2969.6743 - val_loss: 3052.6570 - val_mean_squared_error: 3052.6570
Epoch 320/10000
1/1 [==============================] - 0s 16ms/step - loss: 2967.3611 - mean_squared_error: 2967.3611 - val_loss: 3049.5002 - val_mean_squared_error: 3049.5002
Epoch 321/10000
1/1 [==============================] - 0s 16ms/step - loss: 2965.1001 - mean_squared_error: 2965.1001 - val_loss: 3046.4053 - val_mean_squared_error: 3046.4053
Epoch 322/10000
1/1 [==============================] - 0s 17ms/step - loss: 2962.8901 - mean_squared_error: 2962.8901 - val_loss: 3043.3704 - val_mean_squared_error: 3043.3704
Epoch 323/10000
1/1 [==============================] - 0s 16ms/step - loss: 2960.7305 - mean_squared_error: 2960.7305 - val_loss: 3040.3

Epoch 365/10000
1/1 [==============================] - 0s 15ms/step - loss: 2903.6995 - mean_squared_error: 2903.6995 - val_loss: 2956.4241 - val_mean_squared_error: 2956.4241
Epoch 366/10000
1/1 [==============================] - 0s 16ms/step - loss: 2902.9224 - mean_squared_error: 2902.9224 - val_loss: 2955.1602 - val_mean_squared_error: 2955.1602
Epoch 367/10000
1/1 [==============================] - 0s 16ms/step - loss: 2902.1638 - mean_squared_error: 2902.1638 - val_loss: 2953.9216 - val_mean_squared_error: 2953.9216
Epoch 368/10000
1/1 [==============================] - 0s 16ms/step - loss: 2901.4238 - mean_squared_error: 2901.4238 - val_loss: 2952.7075 - val_mean_squared_error: 2952.7075
Epoch 369/10000
1/1 [==============================] - 0s 16ms/step - loss: 2900.7017 - mean_squared_error: 2900.7017 - val_loss: 2951.5178 - val_mean_squared_error: 2951.5178
Epoch 370/10000
1/1 [==============================] - 0s 16ms/step - loss: 2899.9971 - mean_squared_error: 2899.9971 - 

1/1 [==============================] - 0s 15ms/step - loss: 2882.0630 - mean_squared_error: 2882.0630 - val_loss: 2918.1152 - val_mean_squared_error: 2918.1152
Epoch 412/10000
1/1 [==============================] - 0s 16ms/step - loss: 2881.8135 - mean_squared_error: 2881.8135 - val_loss: 2917.6140 - val_mean_squared_error: 2917.6140
Epoch 413/10000
1/1 [==============================] - 0s 16ms/step - loss: 2881.5706 - mean_squared_error: 2881.5706 - val_loss: 2917.1233 - val_mean_squared_error: 2917.1233
Epoch 414/10000
1/1 [==============================] - 0s 16ms/step - loss: 2881.3328 - mean_squared_error: 2881.3328 - val_loss: 2916.6421 - val_mean_squared_error: 2916.6421
Epoch 415/10000
1/1 [==============================] - 0s 15ms/step - loss: 2881.1008 - mean_squared_error: 2881.1008 - val_loss: 2916.1704 - val_mean_squared_error: 2916.1704
Epoch 416/10000
1/1 [==============================] - 0s 15ms/step - loss: 2880.8748 - mean_squared_error: 2880.8748 - val_loss: 2915.7

Epoch 458/10000
1/1 [==============================] - 0s 15ms/step - loss: 2874.9292 - mean_squared_error: 2874.9292 - val_loss: 2902.7681 - val_mean_squared_error: 2902.7681
Epoch 459/10000
1/1 [==============================] - 0s 15ms/step - loss: 2874.8452 - mean_squared_error: 2874.8452 - val_loss: 2902.5752 - val_mean_squared_error: 2902.5752
Epoch 460/10000
1/1 [==============================] - 0s 15ms/step - loss: 2874.7632 - mean_squared_error: 2874.7632 - val_loss: 2902.3867 - val_mean_squared_error: 2902.3867
Epoch 461/10000
1/1 [==============================] - 0s 15ms/step - loss: 2874.6826 - mean_squared_error: 2874.6826 - val_loss: 2902.2019 - val_mean_squared_error: 2902.2019
Epoch 462/10000
1/1 [==============================] - 0s 15ms/step - loss: 2874.6040 - mean_squared_error: 2874.6040 - val_loss: 2902.0208 - val_mean_squared_error: 2902.0208
Epoch 463/10000
1/1 [==============================] - 0s 16ms/step - loss: 2874.5264 - mean_squared_error: 2874.5264 - 